# Data Loading Tutorial

In [1]:
# Below is code that helps us test the notebooks
# when not testing, we make the save_path a directory called data
# in the scVI root directory, feel free to move anywhere

In [2]:
def allow_notebook_for_test():
    print("Testing the data loading notebook")

test_mode = False
save_path = "data/"
n_epochs = 400

# Feel free to move this to any convenient location
if not test_mode:
    save_path = "../../data"

In [3]:
import anndata
import numpy as np
import scanpy as sc
import scvi

from scvi.dataset import setup_anndata

## Loading Datasets
`scvi` supports the `AnnData` file format. 

This tutorial will go through the following:
1. TLDR: Quickstart
2. Setting up your anndata object for `scvi`
3. Verifying the setup
4. Advanced explanation of `setup_anndata` and `BioDataset`

## TLDR: Quickstart

In [4]:
# get your favorite anndata object
adata = scvi.dataset.pbmcs_10x_cite_seq(save_path = save_path, run_setup_anndata = False)

# run setup_anndata() 
# REMINDER: run filtering and processing before calling setup_anndata()
setup_anndata(adata, #adata should not be a view
              batch_key = 'batch', # key is specific to adata
              labels_key = None, 
              X_layers_key = None,
              protein_expression_obsm_key = 'protein_expression', # only required if using TotalVI
              protein_names_uns_key = None,
              copy = False)

print(adata.uns['scvi_summary_stats']) #double check setup

# Train!
stats = adata.uns['scvi_summary_stats']
vae = scvi.models.VAE(n_input = stats['n_genes'] )
trainer = scvi.inference.UnsupervisedTrainer(
    vae,
    adata,
    train_size=0.90,
    use_cuda=False,
    frequency=5,
)
trainer.train(n_epochs = n_epochs)

#Save and load an already setup anndata without having to setup again!
adata.write('my_anndata.h5ad')
adata = anndata.read('my_anndata.h5ad')

[2020-07-16 18:27:23,683] INFO - scvi.dataset._anndata | Using data from adata.X
[2020-07-16 18:27:23,684] INFO - scvi.dataset._anndata | Using batches from adata.obs["batch"]
[2020-07-16 18:27:23,686] INFO - scvi.dataset._anndata | No label_key inputted, assuming all cells have same label
[2020-07-16 18:27:23,690] INFO - scvi.dataset._anndata | Computing library size prior per batch
[2020-07-16 18:27:24,308] INFO - scvi.dataset._anndata | Using protein expression from adata.obsm['protein_expression']
[2020-07-16 18:27:24,309] INFO - scvi.dataset._anndata | Using protein names from columns of adata.obsm['protein_expression']
[2020-07-16 18:27:24,311] INFO - scvi.dataset._anndata | Successfully registered anndata object containing 10849 cells, 15792 genes, and 2 batches 
Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels', 'protein_expression']
{'n_batch': 2, 'n_cells': 10849, 'n_genes': 15792, 'n_labels': 1, 'n_proteins': 14, 'protein_names': ['CD3_TotalSeqB'

## Setting up your anndata object for `scvi`

### First, load in your anndata. We will defer to the [anndata API](https://anndata.readthedocs.io/) for reading and loading data. 
Here, we load an example dataset from `scanpy`.

In [5]:
import anndata
import numpy as np
import scanpy as sc
import pandas as pd
import scvi

from scvi.dataset import setup_anndata
adata = sc.datasets.pbmc3k()

print(adata)

AnnData object with n_obs × n_vars = 2700 × 32738
    var: 'gene_ids'


### Then run all the necessary filtering and processing on your anndata.

In [6]:
#example processing steps
sc.pp.filter_genes(adata, min_counts = 1)
sc.pp.filter_genes(adata, min_cells = 1)

print(adata)

AnnData object with n_obs × n_vars = 2700 × 16634
    var: 'gene_ids', 'n_counts', 'n_cells'


### When you're done processing and filtering your anndata, simply call the `setup_anndata()` function to get it ready for `scvi`.

`setup_anndata()` takes the following arguments:
- **adata**: anndata.AnnData object with your data in adata.X
- **batch_key**: key in adata.obs for batch information. If None, will assign the same batch to all the data.
- **labels_key**: key in adata.obs for label information. If None, will assign the same label to all the data. 
- **X_layers_key**: if not None, will use the data from adata.layers[X_layers_key] instead of the default adata.X.
- **protein_expression_obsm_key**: key in adata.obsm containing protein expression data. Only required for TotalVI.
- **protein_names_uns_key**: key in adata.uns containing protein names. If None, will use the column names of adata.obsm[protein_expression_obsm_key] if it is a pandas dataframe, else will assign sequential names to proteins. Only relavent but not required for TotalVI.
- **copy**: if True, will return a copy of setup adata. Default: False

`setup_anndata()` will add the following fields to your anndata. See Section 2 and Section 3 for a more advanced explanation. 
- **adata.uns['scvi\_data\_registry']**: dictionary mapping data field used by `scVI` to their respective locations in adata
- **adata.uns['scvi\_summary\_stats']**: dictionary of summary statistics for adata
- **adata.obs['\_local\_l\_mean']**: per batch library size mean
- **adata.obs['\_local\_l\_var']**: per batch library size variance

`setup_anndata()` will respectively add the following fields if no batch_key or labels_key was provided. Or if they were not encoded as integers.
- **adata.obs['\_scvi_labels']**: labels encoded as integers
- **adata.obs['\_scvi_batch']**: batch encoded as integers


#### Since all our data has the same batch and label, we simply call `setup_anndata()` on our anndata. The anndata is now ready for scvi.


In [7]:
setup_anndata(adata)

[2020-07-16 18:27:32,519] INFO - scvi.dataset._anndata | Using data from adata.X
[2020-07-16 18:27:32,520] INFO - scvi.dataset._anndata | No batch_key inputted, assuming all cells are same batch
[2020-07-16 18:27:32,524] INFO - scvi.dataset._anndata | No label_key inputted, assuming all cells have same label
[2020-07-16 18:27:32,591] INFO - scvi.dataset._anndata | Computing library size prior per batch
[2020-07-16 18:27:32,610] INFO - scvi.dataset._anndata | Successfully registered anndata object containing 2700 cells, 16634 genes, and 1 batches 
Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels']


#### If we want to use label and batch information from our anndata, simply pass the corresponding keys to `setup_anndata()`

In [8]:
adata = sc.datasets.pbmc3k()

#assigning cells to random batches as an example (can be string or numbers)
adata.obs['random_batches'] = np.random.randint(0,2,adata.shape[0])

#assigning cells to random labels as an example (can be strings or numbers)
adata.obs['random_labels'] = np.random.randint(0,5,adata.shape[0])

setup_anndata(adata, batch_key = 'random_batches', labels_key = 'random_labels')

[2020-07-16 18:27:32,822] INFO - scvi.dataset._anndata | Using data from adata.X
[2020-07-16 18:27:32,823] INFO - scvi.dataset._anndata | Using batches from adata.obs["random_batches"]
[2020-07-16 18:27:32,824] INFO - scvi.dataset._anndata | Using labels from adata.obs["random_labels"]
[2020-07-16 18:27:32,825] INFO - scvi.dataset._anndata | Computing library size prior per batch
[2020-07-16 18:27:32,843] INFO - scvi.dataset._anndata | Successfully registered anndata object containing 2700 cells, 32738 genes, and 2 batches 
Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels']


#### If we are using TotalVI, we need to pass in protein_expression_obsm_key

In [9]:
adata = sc.datasets.pbmc3k()

#adding random protein expression information
#make dataframe
protein_expression = pd.DataFrame(np.random.randint(0,100,(adata.shape[0],5)), index = adata.obs_names)
adata.obsm['rand_protein_expressions'] = protein_expression

setup_anndata(adata, protein_expression_obsm_key = 'rand_protein_expressions')

[2020-07-16 18:27:33,048] INFO - scvi.dataset._anndata | Using data from adata.X
[2020-07-16 18:27:33,049] INFO - scvi.dataset._anndata | No batch_key inputted, assuming all cells are same batch
[2020-07-16 18:27:33,052] INFO - scvi.dataset._anndata | No label_key inputted, assuming all cells have same label
[2020-07-16 18:27:33,056] INFO - scvi.dataset._anndata | Computing library size prior per batch
[2020-07-16 18:27:33,073] INFO - scvi.dataset._anndata | Using protein expression from adata.obsm['rand_protein_expressions']
[2020-07-16 18:27:33,074] INFO - scvi.dataset._anndata | Using protein names from columns of adata.obsm['rand_protein_expressions']
[2020-07-16 18:27:33,076] INFO - scvi.dataset._anndata | Successfully registered anndata object containing 2700 cells, 32738 genes, and 1 batches 
Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels', 'protein_expression']


### Verifying registration

Here we explain what `setup_anndata()` does under the hood and how you can check that your anndata was properly setup. 

#### First we load in the cortex dataset from `scVI`.

`cortex()` consists of 3,005 mouse cortex cells profiled with the Smart-seq2 protocol, with the addition of UMI. To facilitate comparison with other methods, we use a filtered set of 558 highly variable genes. The `cortex()` exhibits a clear high-level subpopulation struc- ture, which has been inferred by the authors of the original publication using computational tools and annotated by inspection of specific genes or transcriptional programs.

Reference: Zeisel, A. et al. Cell types in the mouse cortex and hippocampus revealed by single-cell rna-seq. Science 347, 1138–1142 (2015). 

In [ ]:
adata_raw = scvi.dataset.cortex(run_setup_anndata = False) #by default, setup_anndata() is called on preloaded datasets before returning

adata = setup_anndata(adata_raw, labels_key = 'labels', copy = True)

[2020-07-16 18:27:33,083] INFO - scvi.dataset._utils | File /Users/galen/scVI/tests/notebooks/data/expression.bin already downloaded
[2020-07-16 18:27:33,084] INFO - scvi.dataset.cortex | Loading Cortex data from /Users/galen/scVI/tests/notebooks/data/expression.bin


#### Looking at adata.uns['scvi\_summary\_stats'] we can verify that the correct number of genes, cells and batches were registered

In [ ]:
print(adata.uns['scvi_summary_stats'])

stats = adata.uns['scvi_summary_stats']
assert stats['n_cells'] == adata.shape[0]
assert stats['n_genes'] == adata.shape[1]
assert stats['n_labels'] == len(np.unique(adata.obs['labels']))
assert stats['n_batch'] == len(np.unique(adata.obs['_scvi_batch'])) # since batch_key was None, setup_anndata put batch info in _scvi_batch

### Advanced explanation
Here we provide an in depth explanation of `setup_anndata()` and how the fields are used under the hood in `scVI`

#### scvi_data_registry hold the mapping between fields needed by `scvi` and their location in anndata

Each field required for `scVI` is mapped to a two element list. 
The first element in the list is an attribute of anndata. The second is the key of the attribute containing the data. 

Eg. the list associated with `labels` is `['obs', 'labels']`. Thus `scVI` grabs label information from `adata.obs['labels']`

If we look at the list associated with `X`, we see that it is : `['_X', None]`. The second element in the list for `X` is empty. That is because `adata._X` (which contains the same data as `adata.X` but has faster accessing) has no key associated with that field. 

In [ ]:
adata_raw = scvi.dataset.cortex(run_setup_anndata = False) #by default, setup_anndata() is called on preloaded datasets before returning
adata = setup_anndata(adata_raw, labels_key = 'labels', copy = True)
adata.uns['scvi_data_registry']

**Looking above, we also see that a number of fields were added to the anndata**

Because no `batch_key` was passed into `setup_anndata()`, all the data was assigned the same batch and saved in `adata.obs['_scvi_batch']`

In [ ]:
print(adata.obs['_scvi_batch'])
print('List of all batches:' , np.unique(adata.obs['_scvi_batch']))

Because the label information in **adata.obs['labels]** is encoded as integers, no `_scvi_labels` was added to `.obs`

In [ ]:
adata.obs['labels']

**When `scVI` is run, it will perform a check on the anndata for the correct number of fields and a simple verification that nothing was changed**

For example, the following throws an error because the anndata was modified after setup:

In [ ]:
import traceback

adata = scvi.dataset.pbmcs_10x_cite_seq(save_path = save_path, run_setup_anndata = False)

setup_anndata(adata, batch_key = 'batch')

adata = adata[:,:10].copy()
stats = adata.uns['scvi_summary_stats']
vae = scvi.models.VAE(n_input = stats['n_genes'] )

#throws error because adata was modified after setup
try:
    trainer = scvi.inference.UnsupervisedTrainer(
        vae,
        adata,
        train_size=0.90,
        use_cuda=False,
        frequency=5,
    )
    trainer.train(n_epochs = n_epochs)
except:
    traceback.print_exc()
